<a href="https://colab.research.google.com/github/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/blob/main/notebooks/04_RNN_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Entrenamiento, Predicción y Evaluación de un modelo RNN

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from datetime import datetime
from io import BytesIO

DATA_GITHUB_URL = 'https://raw.githubusercontent.com/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/refs/heads/main/data/zone1_power_consumption_of_tetouan_city.csv'

# Descargar los datos desde github
response = requests.get(DATA_GITHUB_URL)

# Convertir en un df desde el xlsx de github
df = pd.read_csv(BytesIO(response.content), sep=',')

In [2]:
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")

# Ahora solo quedan las columnas numéricas
df.head()

,temperature,humidity,wind_speed,general_diffuse_flows,zone_1
datetime,,,,,
2017-01-01 00:00:00,6.559,73.8,0.083,0.051,34055.69620
2017-01-01 00:10:00,6.414,74.5,0.083,0.070,29814.68354
2017-01-01 00:20:00,6.313,74.5,0.080,0.062,29128.10127
2017-01-01 00:30:00,6.121,75.0,0.083,0.091,28228.86076
2017-01-01 00:40:00,5.921,75.7,0.081,0.048,27335.69620


In [3]:
# Evaluación
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.abs(y_true - y_pred) / denominator) * 100

def wape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

#### **Entrenamiento**

In [4]:
# Índices por % para train, val, test
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

print(f"Número de características (features): {num_features}")

Número de características (features): 5


In [5]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [6]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                          enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]

        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)
        ds = ds.map(self.split_window)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        result = getattr(self, '_example', None)
        if result is None:
            result = next(iter(self.train))
            self._example = result
        return result

In [7]:
# Predicción a 1 paso (10 minutos adelante)
single_step_window = WindowGenerator(
    input_width=18, label_width=1, shift=1, label_columns=["zone_1"])

# Predicción multi-step (18 pasos = 3 horas adelante)
multi_step_window = WindowGenerator(
    input_width=18, label_width=18, shift=18, label_columns=["zone_1"])

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

def compile_and_fit(model, window, patience=3, epochs=20):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])
    history = model.fit(window.train, validation_data=window.val,
                        epochs=epochs, callbacks=[early_stopping])
    return history

---

#### **Predicción**

A un paso

In [9]:
# Ventana single-step
single_step_window = WindowGenerator(
    input_width=18, label_width=1, shift=1,
    label_columns=["zone_1"]
)

# Modelo con SimpleRNN
simple_rnn_single = Sequential([
    SimpleRNN(64, return_sequences=True),    # recurrente simple
    Dense(units=1)                           # salida 1 paso adelante
])

# Entrenamiento
history_single = compile_and_fit(simple_rnn_single, single_step_window)

# Evaluación
val_single = simple_rnn_single.evaluate(single_step_window.val, return_dict=True)
test_single = simple_rnn_single.evaluate(single_step_window.test, return_dict=True)

print("SimpleRNN (single-step) Validation:", val_single)
print("SimpleRNN (single-step) Test:", test_single)

Epoch 1/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - loss: 0.2785 - mean_absolute_error: 0.3952 - val_loss: 0.1793 - val_mean_absolute_error: 0.3169
Epoch 2/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.1681 - mean_absolute_error: 0.3007 - val_loss: 0.1675 - val_mean_absolute_error: 0.3027
Epoch 3/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.1485 - mean_absolute_error: 0.2763 - val_loss: 0.1685 - val_mean_absolute_error: 0.3059
Epoch 4/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.1406 - mean_absolute_error: 0.2670 - val_loss: 0.1945 - val_mean_absolute_error: 0.3164
Epoch 5/20
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.1338 - mean_absolute_error: 0.2584 - val_loss: 0.1935 - val_mean_absolute_error: 0.3139
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1946 - mean_absolute_error: 0.3154
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1516 - mean_absolute_error: 0.2857
SimpleRNN (single-step) Validation: {'loss': 0.19351369142532

A múltiples pasos

In [10]:
# Ventana multi-step (18 pasos = 3 horas al futuro)
multi_step_window = WindowGenerator(
    input_width=18, label_width=18, shift=18,
    label_columns=["zone_1"]
)

simple_rnn_multi = Sequential([
    SimpleRNN(64, return_sequences=False),   # ahora solo última salida
    Dense(18),                               # salida: 18 pasos adelante
    tf.keras.layers.Reshape([18, 1])         # dar forma correcta (batch, 18, 1)
])

# Entrenamiento
history_multi = compile_and_fit(simple_rnn_multi, multi_step_window)

# Evaluación
val_multi = simple_rnn_multi.evaluate(multi_step_window.val, return_dict=True)
test_multi = simple_rnn_multi.evaluate(multi_step_window.test, return_dict=True)

print("SimpleRNN (multi-step) Validation:", val_multi)
print("SimpleRNN (multi-step) Test:", test_multi)

Epoch 1/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.2374 - mean_absolute_error: 0.3547 - val_loss: 0.1408 - val_mean_absolute_error: 0.2775
Epoch 2/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.1065 - mean_absolute_error: 0.2323 - val_loss: 0.1502 - val_mean_absolute_error: 0.2749
Epoch 3/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.0925 - mean_absolute_error: 0.2129 - val_loss: 0.1161 - val_mean_absolute_error: 0.2378
Epoch 4/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0795 - mean_absolute_error: 0.1956 - val_loss: 0.1089 - val_mean_absolute_error: 0.2334
Epoch 5/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0781 - mean_absolute_error: 0.1921 - val_loss: 0.1030 - val_mean_absolute_error: 0.2231
Epoch 6/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0704 - mean_absolute_error: 0.1814 - val_loss: 0.1320 - val_mean_absolute_error: 0.2521
Epoch 7/20
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0667 - mean_abso

---

#### **Analisis de resultados**

Métricas a evaluar:
1. sMAPE
2. WAPE

In [11]:
# Evaluación con métricas personalizadas en multi-step
y_true, y_pred = [], []

for x, y in multi_step_window.test.take(10):
    preds = simple_rnn_multi.predict(x)
    y_true.extend(y.numpy().reshape(-1))
    y_pred.extend(preds.reshape(-1))

print("SMAPE:", smape(y_true, y_pred))
print("WAPE:", wape(y_true, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
SMAPE: 52.273613
WAPE: 25.361681


---